# Data cleaning

Este documento nos será de utilidad para poder introducir nuevos dataset que serán usados como test de los modelos.

## Eliminación de variables que fueron excluidas durante la ingeniería de variables y el EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = ('../data/02_intermediate/loan_all.csv') # aqui introducimos el dataset en cuestión
data = pd.read_csv(path, low_memory=False)

In [3]:
# Eliminamos las variables con más del 95% de NAS en el dataset inicial

del data['member_id']
del data['desc']
del data['annual_inc_joint'] 
del data['dti_joint']                 
del data['verification_status_joint']                     
del data['revol_bal_joint']                               
del data['sec_app_fico_range_low']                       
del data['sec_app_fico_range_high']                       
del data['sec_app_earliest_cr_line']                      
del data['sec_app_inq_last_6mths']                        
del data['sec_app_mort_acc']                              
del data['sec_app_open_acc']                              
del data['sec_app_revol_util']                            
del data['sec_app_open_act_il']                           
del data['sec_app_num_rev_accts']                         
del data['sec_app_chargeoff_within_12_mths']              
del data['sec_app_collections_12_mths_ex_med']            
del data['sec_app_mths_since_last_major_derog']           
del data['hardship_type']                                 
del data['hardship_reason']                               
del data['hardship_status']                               
del data['deferral_term']                                 
del data['hardship_amount']                               
del data['hardship_start_date']                           
del data['hardship_end_date']                             
del data['payment_plan_start_date']                       
del data['hardship_length']                               
del data['hardship_dpd']                                 
del data['hardship_loan_status']                          
del data['orig_projected_additional_accrued_interest']    
del data['hardship_payoff_balance_amount']                
del data['hardship_last_payment_amount']                  
del data['debt_settlement_flag_date']

In [4]:
# Eliminamos la columna unnamed
del data['Unnamed: 0']
# Variables que omitimos por su contenido:
del data['out_prncp_inv']
del data['total_pymnt']
del data['out_prncp']
del data['total_pymnt_inv']
del data['total_rec_prncp']
del data['recoveries']
del data['hardship_flag']
del data['debt_settlement_flag']
del data['settlement_date']
del data['settlement_status']
del data['settlement_amount']
del data['settlement_percentage']
del data['settlement_term']
del data['emp_title']
del data['issue_d']
del data['last_credit_pull_d']
del data['last_pymnt_d']
del data['total_rec_int']
del data['total_rec_late_fee']
del data['last_pymnt_amnt']
del data['pymnt_plan']
del data['next_pymnt_d']

In [5]:
# variables que no se pueden usar para predecir el impago
del data['url']
del data['title']
del data['zip_code']
del data['addr_state']
# variables con excesivo número de clases
del data['sub_grade']
del data['earliest_cr_line']

In [6]:
# Categóricas problemáticas por distintas razones
del data['disbursement_method']
del data['initial_list_status']

## Definición de la variable target

In [7]:
# Eliminamos las observaciones cuyo status es "Current"
data = data.drop(data[data['loan_status']=="Current"].index)

In [8]:
# Creamos columna llamada target
# toma valor 0 si fully paid y valor 1 en caso contrario (impago)
data['target'] = np.where((data['loan_status'] == 'Fully Paid'), 0, 1)

In [9]:
# Eliminamos la columna de loan_status y dejamos solo la columna de Target
del data['loan_status']

## Sustituimos los Valores Nulos

### Variables con valores nulos por razones alternativas

Rellenamos con cero aquellas que los usuarios no hayan realizado la acción de la variable

In [10]:
# Definimos función para rellenar 0 los NAs 
def fill_w_0(data, column):
    data[column].fillna(0, inplace = True)

In [11]:
# refilling with 0
fill_w_0(data, 'il_util')
fill_w_0(data, 'all_util')
fill_w_0(data, 'inq_last_12m')
fill_w_0(data, 'total_cu_tl')
fill_w_0(data, 'open_acc_6m')
fill_w_0(data, 'open_act_il')
fill_w_0(data, 'inq_fi')
fill_w_0(data, 'max_bal_bc')
fill_w_0(data, 'open_rv_12m')
fill_w_0(data, 'open_rv_24m')
fill_w_0(data, 'total_bal_il')
fill_w_0(data, 'open_il_12m')
fill_w_0(data, 'open_il_24m')

Rellenamos con el máximo las variables relativas a meses

In [12]:
# Definimos función para rellenar con el máximo los NAs
def fill_max(data, column):
    data[column].fillna(data[column].max(), inplace = True)

In [13]:
# refilling with the maximum

fill_max(data, 'mths_since_last_record')
fill_max(data, 'mths_since_recent_bc_dlq')
fill_max(data, 'mths_since_recent_revol_delinq')
fill_max(data, 'mths_since_last_major_derog')
fill_max(data, 'mths_since_rcnt_il')
fill_max(data, 'mths_since_last_delinq')
fill_max(data, 'mths_since_recent_inq')

### Categóricas

Creamos dataframe solo con las categóricas.

In [14]:
categoric_1 = data.select_dtypes(include = [np.object])

Sustituimos por la moda las variables categóricas

In [15]:
categoric_1 = categoric_1.apply(lambda x: x.replace("", np.nan))
categoric_1 = categoric_1.apply(lambda x: x.fillna(x.value_counts().index[0])) # fill with the mode

In [16]:
# En principio, todas las variables que son categóricas en nuestro dataframe, lo serán en el del día de prueba.

### Variables numéricas

Creamos un dataframe con las variables numéricas  y sustituimos los valores nulos por la mediana

In [17]:
numeric = data.select_dtypes(include = [np.number])

In [18]:
numeric = numeric.apply(lambda x: x.replace("", np.nan))
numeric = numeric.apply(lambda x: x.fillna(x.median()))

## Transformación de variables categóricas

Reducimos las clases de la variable emp_length 

In [19]:
def f_emp_length(x):
    if x == '< 1 year':
        return '< 5 years'
    elif x == '1 year':
        return '< 5 years'
    elif x == '2 years':
        return '< 5 years'
    elif x == '3 years':
        return '< 5 years'
    elif x == '5 years':
        return '5-10 years'
    elif x == '6 years':
        return '5-10 years'
    elif x == '7 years':
        return '5-10 years'
    elif x == '8 years':
        return '5-10 years'
    elif x == '9 years':
        return '5-10 years'
    else:
        return '> 10 years'
    
categoric_1.emp_length = categoric_1.emp_length.map(f_emp_length)

Reducción de categorías de la variable purpose

In [20]:
# creamos la categoría housing
categoric_1['purpose'] = np.where ((categoric_1['purpose'] == 'home_improvement') | 
                            (categoric_1['purpose'] == 'house') | 
                            (categoric_1['purpose'] == 'moving'), 
                           'housing', categoric_1['purpose'].values)

In [21]:
# creamos la categoría leisure
categoric_1['purpose'] = np.where ((categoric_1['purpose'] == 'major_purchase') | 
                            (categoric_1['purpose'] == 'vacation') | 
                            (categoric_1['purpose'] == 'wedding') |
                            (categoric_1['purpose'] == 'car')|
                            (categoric_1['purpose'] == 'small_business') | 
                            (categoric_1['purpose'] == 'renewable_energy'), 
                           'leisure', categoric_1['purpose'].values)

In [22]:
# creamos la categoría otros
categoric_1['purpose'] = np.where ((categoric_1['purpose'] == 'other') | 
                            (categoric_1['purpose'] == 'educational'), 
                           'other', categoric_1['purpose'].values)

Transformamos la variable grade

In [23]:
# Creamos un diccionario para cada letra:
dic =  {'A': 6, 'B': 5, 'C': 4, 'D': 3, 'E': 2, 'F': 1, 'G': 0} 

# Sustituimos en cada caso su valor del diccionario:

categoric_1['grade'] = categoric_1['grade'].replace(dic)

## Cambio formato de variables con porcentajes

Eliminamos el porcentaje de la variable int_rate y convertimos en decimal.

In [24]:
# Convertimos a string
int_rate_conversion = categoric_1['int_rate']
int_rate_conversion = int_rate_conversion.astype(str) 

In [25]:
# Eliminamos el símbolo %
int_rate_conversion = int_rate_conversion.map(lambda x: x[:-1])

In [26]:
# Convertimos a decimal dividiendo entre 100
int_rate_conversion= int_rate_conversion.astype(float)/100

In [27]:
# Introducimos en el dataset y eliminamos el objeto creado int_rate_conversion
categoric_1['int_rate'] = int_rate_conversion 
del int_rate_conversion

Eliminamos el porcentaje de la variable revol_util y convertimos en decimal.

In [28]:
# Convertimos a string
revol_util_conversion = categoric_1['revol_util']
revol_util_conversion = revol_util_conversion.astype(str)

In [29]:
# Eliminamos el %
revol_util_conversion = revol_util_conversion.map(lambda x: x[:-1])

In [30]:
# Convertimos en decimal
revol_util_conversion= revol_util_conversion.astype(float)/100

In [31]:
# Introducimos en el dataset y eliminamos el objeto creado revol_util_conversion
categoric_1['revol_util'] = revol_util_conversion 
del revol_util_conversion

## One hot-encoding

Primero creamos un dataframe que junte las variables del dataframe categoric con el de numeric.

In [32]:
data = pd.concat([numeric, categoric_1], axis = 1)

Creamos un dataframe solo con las que son categóricas (ya que en el anterior estaban dos variables que tuvimos que transformar)

In [33]:
categoric_2 = data.select_dtypes(include = [np.object])
categoric_2.head()

,term,emp_length,home_ownership,verification_status,purpose,application_type
8,36 months,5-10 years,OWN,Not Verified,credit_card,Individual
10,36 months,< 5 years,RENT,Source Verified,debt_consolidation,Individual
24,36 months,5-10 years,MORTGAGE,Not Verified,debt_consolidation,Individual
42,60 months,5-10 years,RENT,Source Verified,debt_consolidation,Individual
91,36 months,5-10 years,MORTGAGE,Verified,debt_consolidation,Individual


Creamos variables dummies

In [34]:
dummies = pd.get_dummies(categoric_2)
dummies.head()

,term_ 36 months,term_ 60 months,emp_length_5-10 years,emp_length_< 5 years,emp_length_> 10 years,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_housing,purpose_leisure,purpose_medical,purpose_other,application_type_Individual,application_type_Joint App
8,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0
10,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0
24,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0
42,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0
91,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0


Eliminamos las variables del dataset data para que al juntar dummies con numeric no se duplicen.

In [35]:
data_filter = pd.merge(numeric, dummies, left_index=True, right_index=True)

In [36]:
data_filter.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_housing,purpose_leisure,purpose_medical,purpose_other,application_type_Individual,application_type_Joint App
8,112038251,11575,11575,11575.0,359.26,153000.0,16.99,0,720,724,...,0,0,1,0,0,0,0,0,1,0
10,112149045,7200,7200,7200.0,285.70,50000.0,6.07,0,685,689,...,1,0,0,1,0,0,0,0,1,0
24,112052261,7500,7500,7500.0,232.79,110000.0,13.12,0,710,714,...,0,0,0,1,0,0,0,0,1,0
42,111999259,10000,10000,10000.0,243.29,51979.0,10.11,0,690,694,...,1,0,0,1,0,0,0,0,1,0
91,111808508,14000,14000,14000.0,492.34,75000.0,10.86,1,685,689,...,0,1,0,1,0,0,0,0,1,0


In [37]:
data_filter.shape

(462191, 99)

## Elección de las variables mediante lasso

In [39]:
final_df = data_filter.loc[:, ['num_sats',
 'home_ownership_RENT',
 'mo_sin_rcnt_rev_tl_op',
 'funded_amnt_inv',
 'open_rv_12m',
 'num_rev_tl_bal_gt_0',
 'acc_open_past_24mths',
 'total_cu_tl',
 'open_acc',
 'collection_recovery_fee',
 'percent_bc_gt_75',
 'num_tl_op_past_12m',
 'policy_code',
 'il_util',
 'total_bal_il',
 'last_fico_range_low',
 'application_type_Individual',
 'acc_now_delinq',
 'total_acc',
 'verification_status_Source Verified',
 'total_bal_ex_mort',
 'mo_sin_old_il_acct',
 'tot_cur_bal',
 'num_bc_tl',
 'total_bc_limit',
 'emp_length_< 5 years',
 'mths_since_last_record',
 'num_actv_rev_tl',
 'inq_last_12m',
 'mort_acc',
 'open_act_il',
 'id',
 'all_util',
 'revol_bal',
 'num_tl_90g_dpd_24m',
 'home_ownership_OWN',
 'mths_since_recent_revol_delinq',
 'total_rev_hi_lim',
 'mths_since_rcnt_il',
 'delinq_amnt',
 'application_type_Joint App',
 'purpose_housing',
 'total_il_high_credit_limit',
 'open_il_12m',
 'target']]

In [ ]:
ruta = '../data/02_intermediate/test_final_df.csv'

In [ ]:
final_df.to_csv(ruta)

## Escalado de variables

In [ ]:
scaler = MinMaxScaler()  # escalamos los datos                                     
df_scaled = pd.DataFrame(scaler.fit_transform(final_df), columns = final_df.columns) # trasformamos a df los datos escalados
df_scaled.head()

In [ ]:
X_test = df_scaled.loc[:, df_scaled.columns != 'target'] #serán todas menos la varoables target

Y_test = df_scaled['target']  